In [ ]:
import pandas as pd
import numpy as np
from funciones import *

# Configuracion de display de DataFrames en Jupyter
pd.options.display.max_columns = None

# Localizacion del .CSV del dataset
data_location_raw = './datasets/properatti.csv'

# Localizacion del .CSV donde enviaremos los datos limpios
data_location_clean = './datasets/properatti_clean.csv'

# Hacemos la importación del dataframe clean
raw_data = pd.read_csv(data_location_raw, index_col=0)

# Copiamos el dataframe a otro para tener una copia literal del mismo
data = raw_data.copy()

# Forma de resetear el index del dataframe si no se usa el index_col=0
## data_default.reset_index(drop=True, inplace=True)
## data_str.index

Información del DataFrame RAW que hemos importado

In [ ]:
raw_data.info()

Información del DataFrame que vamos a utilizar para nuestro análisis

In [ ]:
data.info()

Dataset

#Completar con nombre de campo y tipo de dato

### Sumario inicial de datos faltantes

In [ ]:
missing_zero_values_table(data)

In [ ]:
data.head(10)

### Análisis del campo _description_

In [ ]:
mask_description_isnull = data['description'].isnull()

print(f"Cantidad de observaciones sin descripción: {mask_description_isnull.sum()}")

data.loc[mask_description_isnull, 'description'] = '-'

missing_zero_values_table(data)

In [ ]:
data.head(10)

### Análisis de la variable _operation_

In [ ]:
data['operation'].value_counts()

Todas las propiedades son para la venta, lo cual representa una categoría única.

In [ ]:
data['operation'] = data['operation'].astype('category')
data['operation'].dtype

### Análisis de la variable _property_type_

In [ ]:
data['property_type'].value_counts()

Observamos que tiene tiene cuatro posibles opciones por lo cual la cambiamos a categórica.

In [ ]:
data['property_type'] = data['property_type'].astype('category')
data['property_type'].dtype

### Análisis de la variable _place_name_

In [ ]:
data['place_name'].value_counts()

Haciendo un análisis de las 23 observaciones que no tienen 'place_name', observamos que las mismas pertenecen al barrio de Rincón de Mimberg, y hay observaciones que tienen las mismas características para descripciones similares. Sumado a que geográficamente las descripciones apuntan a que se encuentran en barrios cerrados ó courtries en dicha ubicación.

In [ ]:
mask_data_place_name_isnull = data['place_name'].isnull()
data_place_name_null = data.loc[mask_data_place_name_isnull]
data_place_name_null.head(5)

In [ ]:
data[data['place_name'].str.contains('Rincón de Milberg',na=False)].head(5)

Por este motivo imputamos a los datos faltantes con Rincón de Milberg, ajustando al mismo tiempo el campo place_with_parent_names para que sea consistente con la información del dataset.

In [ ]:
#data['place_name_check'] = data['place_name']
#data['place_with_parent_names_check'] = data['place_with_parent_names']

data.loc[mask_data_place_name_isnull,['place_name','place_with_parent_names']] = \
    ['Rincón de Milberg','|Argentina|Bs.As. G.B.A. Zona Norte|Tigre|Rincón de Milberg|']
data.head(5)

Detectamos que existe un place_name que está cargado con coordenadas en lugar de nombre

In [ ]:
data[data['place_name'].str.contains('coor',na=False)]

Observando en Google Maps vemos que también es el barrio Rincon de Milberg, con lo cual volvemos a ejecutar la operación anterior, pero con esta nueva máscara.

In [ ]:
data.loc[data['place_name'].str.contains('coor',na=False),['place_name','place_with_parent_names']] = \
    ['Rincón de Milberg','|Argentina|Bs.As. G.B.A. Zona Norte|Tigre|Rincón de Milberg|']
data

### Análisis de la variable _country_name_

In [ ]:
data['country_name'].value_counts()

No es necesario realizar ningún tipo de ajuste en esta variable. Vemos que todas las propiedades son en Argentina.

### Análisis de la variable _state_name_

In [ ]:
data['state_name'].value_counts()

No es necesario realizar ningún tipo de ajuste en esta variable. Observamos que la provinicia de Buenos Aires ha sido dividida en regiones geográficas, a diferencia del resto de las provincias.

In [ ]:
missing_zero_values_table(data)

### Exportación de datos limpios

In [ ]:
data.to_csv(data_location_clean)